<a href="https://colab.research.google.com/github/zahran1122/RegularizationGANs/blob/main/RegularizationGANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularization GANs

To run this code, we need to have CelebA sample. First, download the celebA file. Second, create new folder in the colab files and name it "sample". Upload all the images from CelebA to the "sample" folder. after that, we can run the code.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, Flatten, Dropout, LeakyReLU, BatchNormalization, Activation, UpSampling2D, Conv2D, Input
from keras.optimizers import Adam
import tensorflow as tf

# Function to load and preprocess real images from the given directory
def load_real_samples(directory, image_size=(64, 64)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=image_size)
            img_array = img_to_array(img)
            images.append(img_array)
    images = np.array(images, dtype='float32')
    images = (images - 127.5) / 127.5
    return images

# Function to build the generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128 * 16 * 16, activation="relu", input_dim=latent_dim))
    model.add(Reshape((16, 16, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=4, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=4, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(3, kernel_size=4, padding="same"))
    model.add(Activation("tanh"))
    return model

# Function to build the discriminator model
def build_discriminator(image_shape):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=4, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=4, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

# Function to build the GAN model by combining the generator and the discriminator
def build_gan(generator, discriminator, latent_dim):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    img = generator(gan_input)
    gan_output = discriminator(img)
    gan = Model(gan_input, gan_output)
    return gan

# Function to save the generated images
def save_images(generator, epoch, latent_dim, n=5):
    noise = np.random.normal(0, 1, (n * n, latent_dim))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(n, n)
    cnt = 0
    for i in range(n):
        for j in range(n):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    if not os.path.exists('images1'):
        os.makedirs('images1')
    fig.savefig(f"images1/gan_images_epoch_{epoch}.png")
    plt.close()

# Function to train the GAN model
def train_gan_with_mode_seeking(generator, discriminator, gan, dataset, latent_dim, epochs=1000, batch_size=64, save_interval=1, lambda_ms=0.1):
    half_batch = int(batch_size / 2)

    for epoch in range(epochs):
        # Train the discriminator
        idx = np.random.randint(0, dataset.shape[0], half_batch)
        real_imgs = dataset[idx]

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.array([1] * batch_size)

        g_loss = gan.train_on_batch(noise, valid_y)

        # Mode-seeking regularization term
        z1 = np.random.normal(0, 1, (half_batch, latent_dim))
        z2 = np.random.normal(0, 1, (half_batch, latent_dim))
        g1 = generator.predict(z1)
        g2 = generator.predict(z2)

        # Compute distances
        d_I = np.mean(np.abs(g1 - g2), axis=(1, 2, 3))
        d_z = np.mean(np.abs(z1 - z2), axis=1)

        # Compute L_ms
        L_ms = np.mean(d_I / (d_z + 1e-8))

        # Update generator loss with L_ms
        g_loss_with_reg = g_loss + lambda_ms * L_ms

        # Print the progress
        print(f"{epoch + 1} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]:.2f}%] [G loss: {g_loss_with_reg}] [L_ms: {L_ms}]")

        if (epoch + 1) % save_interval == 0:
            save_images(generator, epoch + 1, latent_dim)

# Set parameters
image_shape = (64, 64, 3)
latent_dim = 100

# Load the dataset
dataset = load_real_samples('/content/sample')

# Build and compile the models
optimizer = tf.keras.optimizers.legacy.Adam(0.0002, 0.5)

discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

generator = build_generator(latent_dim)

gan = build_gan(generator, discriminator, latent_dim)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

# Train the GAN model with mode-seeking regularization
train_gan_with_mode_seeking(generator, discriminator, gan, dataset, latent_dim, epochs=1000, batch_size=64, save_interval=1, lambda_ms=0.1)


# Convergence Graphs



> CONVERGENCE GRAPHS



To implement this code, first run the Regularization GANs code for one time. Copy the output and paste it to notepad name it "real.text". Upload the file. Convergence graph will be shown.

**This is code for Plot of Loss Functions and Regularization term Values**

In [ ]:
import re
import matplotlib.pyplot as plt

# Reading the data from the file
file_path = '/content/real.txt'

# Initialize lists to store the data
epochs = []
d_losses = []
g_losses = []
l_ms_values = []

# Regex pattern to extract the required values
pattern = re.compile(r'(\d+)\s\[D loss: ([\d.]+),.*?\[G loss: ([\d.]+)\].*?\[L_ms: ([\d.]+)\]')

# Read and parse the file
with open(file_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = pattern.search(line)
        if match:
            epoch = int(match.group(1))
            d_loss = float(match.group(2))
            g_loss = float(match.group(3))
            l_ms = float(match.group(4))

            epochs.append(epoch)
            d_losses.append(d_loss)
            g_losses.append(g_loss)
            l_ms_values.append(l_ms)

# Plotting the combined graph
plt.figure(figsize=(10, 6))
plt.plot(epochs, d_losses, label='D_loss')
plt.plot(epochs, g_losses, label='G_loss')
# plt.plot(epochs, l_ms_values, label='L_ms')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Convergence Graph')
# plt.tight_layout()
plt.legend()
plt.savefig('/content/convergence_combined.png')
plt.show()

# L_ms
# Plotting the combined graph
plt.figure(figsize=(10, 6))
# plt.plot(epochs, d_losses, label='D_loss')
# plt.plot(epochs, g_losses, label='G_loss')
plt.plot(epochs, l_ms_values, label='L_ms')
plt.xlabel('Epoch')
plt.ylabel('L_ms Value')
plt.title('Convergence Graph')
# plt.tight_layout()
plt.legend()
plt.savefig('/content/convergence_combined.png')
plt.show()

# Plotting individual graphs
fig, axs = plt.subplots(3, 1, figsize=(10, 18))

axs[2].plot(epochs, l_ms_values, color='red')
axs[2].set_title('L_ms')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('L_ms Value')
plt.tight_layout()
# D_loss
axs[0].plot(epochs, d_losses, color='blue')
axs[0].set_title('D_loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')

# G_loss
axs[1].plot(epochs, g_losses, color='green')
axs[1].set_title('G_loss')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].grid(True)

**This is code for Plots of Discriminator Real and Fake Accuracy**

In [ ]:
import re
import matplotlib.pyplot as plt

# 读取数据文件
file_path = '/content/real.txt'

# 初始化存储数据的列表
epochs = []
real_accuracies = []
fake_accuracies = []

# 提取所需值的正则表达式模式
pattern = re.compile(r'(\d+)\s\[D loss: [\d.]+, acc.: ([\d.]+)%\].*?\[G loss: [\d.]+\].*?\[L_ms: [\d.]+\]')

# 读取并解析文件
with open(file_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = pattern.search(line)
        if match:
            epoch = int(match.group(1))
            d_acc = float(match.group(2))

            # Assuming real accuracy and fake accuracy are complementary
            real_acc = d_acc
            fake_acc = 100 - d_acc

            epochs.append(epoch)
            real_accuracies.append(real_acc)
            fake_accuracies.append(fake_acc)

# 绘制真实和伪造数据的准确性收敛图
plt.figure(figsize=(10, 6))
plt.plot(epochs, real_accuracies, label='Real Accuracy')
plt.plot(epochs, fake_accuracies, label='Fake Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Plots of Accuracy')
plt.legend()
plt.grid(True)
plt.savefig('/content/real_fake_accuracy_convergence.png')
plt.show()

# 显示生成的图像文件路径
output_files = {
    "real_fake_accuracy_convergence": "/content/real_fake_accuracy_convergence.png"
}

output_files


# FID Score

To implement this code, first run the Regularization GANs code for one time. Copy the output and paste it to notepad. run this code and upload the run.txt. FID score will be shown.

In [ ]:
import re
import pandas as pd
from google.colab import files
from tabulate import tabulate

# Define a function to extract FID scores and epochs
def extract_fid_scores(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Use regex to find all FID scores and corresponding epochs in the content
    matches = re.findall(r'FID score at epoch (\d+): ([\d.]+)', content)
    epochs = [int(match[0]) for match in matches]
    fid_scores = [float(match[1]) for match in matches]

    return epochs, fid_scores

# Define a function to calculate the mean FID score from a list of scores
def calculate_mean_fid(scores):
    return sum(scores) / len(scores)

# Upload the file
uploaded = files.upload()

# Assuming the file is named 'FID-Regularization.txt'
file_path = list(uploaded.keys())[0]

# Extract FID scores
epochs, fid_scores = extract_fid_scores(file_path)

# Filter scores for epochs 100 to 1000
filtered_epochs = [epoch for epoch in epochs if 100 <= epoch <= 1000]
filtered_fid_scores = [fid_scores[i] for i, epoch in enumerate(epochs) if 100 <= epoch <= 1000]

# Create a DataFrame to tabulate the values
df = pd.DataFrame({
    'Epoch': filtered_epochs,
    'FID Score': filtered_fid_scores
})

# Calculate the mean and standard deviation of FID scores
mean_fid_score = calculate_mean_fid(filtered_fid_scores)
std_fid_score = pd.Series(filtered_fid_scores).std()

# Display the table in a prettier format using tabulate
print("FID Scores from Epoch 100 to 1000:")
print(tabulate(df, headers='keys', tablefmt='pretty'))

# Display the mean and standard deviation
print(f'\nMean FID score from epoch 100 to 1000: {mean_fid_score:.2f}')
print(f'Standard Deviation of FID score from epoch 100 to 1000: {std_fid_score:.2f}')
